## Building Out Gold Data

In [0]:
VOLUME_ROOT_PATH = "/Volumes/cscie103_catalog/final_project"
VOLUME_DATA_DIR = f"{VOLUME_ROOT_PATH}/data"

CATALOG_NAME = "cscie103_catalog"
SCHEMA_NAME = "final_project"
spark.sql(f"USE {CATALOG_NAME}.{SCHEMA_NAME}")

class DataframeNames:
    HOLIDAYS = "holidays"
    OIL = "oil"
    STORES = "stores"
    TEST = "test"
    TRAIN = "train"
    TRANSACTIONS = "transactions"
    TRAINING = "training"

    ALL = [ HOLIDAYS, OIL, STORES, TEST, TRAIN, TRANSACTIONS, TRAINING ]

class DataTier:
    BRONZE = "bronze"
    SILVER = "silver"
    GOLD = "gold"

    def getBronzeName(tablename):
        return DataTier.BRONZE + "_" + tablename

    def getSilverName(tablename):
        return DataTier.SILVER + "_" + tablename
    
    def getGoldName(tablename):
        return DataTier.GOLD + "_" + tablename

In [0]:
from pyspark.sql import functions as F

# Use the right catalog & schema
spark.sql("USE cscie103_catalog.final_project")

# Load Silver tables
silver_train  = spark.table(DataTier.getSilverName(DataframeNames.TRAIN))
silver_stores = spark.table(DataTier.getSilverName(DataframeNames.STORES))
silver_oil    = spark.table(DataTier.getSilverName(DataframeNames.OIL))
silver_hol    = spark.table(DataTier.getSilverName(DataframeNames.HOLIDAYS))
silver_tx     = spark.table(DataTier.getSilverName(DataframeNames.TRANSACTIONS))

# Enriched base fact: one row per date-store-family
base_fact = (
    silver_train.alias("t")
    .join(silver_stores.alias("s"), "store_nbr", "left")
    .join(silver_tx.alias("x"), ["date", "store_nbr"], "left")
    .join(silver_oil.alias("o"), "date", "left")
    .join(silver_hol.alias("h"), "date", "left")
    .select(
        F.col("t.date").alias("date"),
        F.col("t.store_nbr").alias("store_nbr"),
        F.col("t.family").alias("family"),
        F.col("t.sales").alias("sales"),
        F.col("t.onpromotion").alias("onpromotion"),
        F.col("x.transactions").alias("transactions"),
        F.col("o.dcoilwtico").alias("dcoilwtico"),
        F.col("s.city").alias("city"),
        F.col("s.state").alias("state"),
        F.col("s.type").alias("store_type"),
        F.col("s.cluster").alias("cluster"),
        F.col("h.is_holiday").alias("is_holiday")
    )
)

# Databricks notebook: 03_gold_daily_fact
# Default language: Python

# =========================================
# 1. Setup
# =========================================
from pyspark.sql import functions as F

catalog = "cscie103_catalog"
schema  = "final_project"

spark.sql(f"USE {catalog}.{schema}")

# =========================================
# 2. Load Silver tables
# =========================================
silver_train  = spark.table("silver_train")            # date, store_nbr, family, sales, onpromotion, ...
silver_stores = spark.table("silver_stores")           # store_nbr, city, state, cluster, type, ...
silver_oil    = spark.table("silver_oil")              # date, dcoilwtico
silver_hol    = spark.table("silver_holidays_events")  # date, is_holiday, ...
silver_tx     = spark.table("silver_transactions")     # date, store_nbr, transactions

# =========================================
# 3. Build base_fact at date–store–family grain
# =========================================

# Normalize is_holiday safely to 0/1 int
h_str = F.col("h.is_holiday").cast("string")
is_holiday_expr = (
    F.when(h_str.isin("True", "true", "1"), F.lit(1))
     .when(h_str.isin("False", "false", "0"), F.lit(0))
     .otherwise(F.lit(0))
     .cast("int")
     .alias("is_holiday")
)

base_fact = (
    silver_train.alias("t")
    # Add store attributes (city/state/cluster)
    .join(silver_stores.alias("s"), "store_nbr", "left")
    # Daily store-level transactions
    .join(silver_tx.alias("x"), ["date", "store_nbr"], "left")
    # Oil price by date
    .join(silver_oil.alias("o"), "date", "left")
    # Holidays by date
    .join(silver_hol.alias("h"), "date", "left")
    .select(
        F.col("t.id").alias("id"),
        F.col("t.date").alias("date"),
        F.col("t.store_nbr").alias("store_nbr"),
        F.col("t.family").alias("family"),
        F.col("t.sales").cast("double").alias("sales"),
        F.col("t.onpromotion").cast("int").alias("onpromotion"),
        F.col("x.transactions").cast("int").alias("transactions"),
        F.col("o.dcoilwtico").cast("double").alias("dcoilwtico"),
        F.col("s.city").alias("city"),
        F.col("s.state").alias("state"),
        F.col("s.type").alias("store_type"),
        F.col("s.cluster").alias("cluster"),
        is_holiday_expr
    )
)

# =========================================
# 3b. Add core time features
# =========================================
base_fact = (
    base_fact
    .withColumn("year", F.year("date"))
    .withColumn("month", F.month("date"))
    .withColumn("day", F.dayofmonth("date"))
    .withColumn("week_of_year", F.weekofyear("date"))
    .withColumn("day_of_week", F.dayofweek("date"))              # 1=Sun, 7=Sat
    .withColumn("is_weekend", F.col("day_of_week").isin(1, 7).cast("int"))
)

display(base_fact.limit(20))
print("Rows in base_fact:", base_fact.count())

# =========================================
# 4. Save as Gold table: gold_daily_store_family
# =========================================

# Drop if exists to avoid schema merge issues during development
spark.sql("DROP TABLE IF EXISTS gold_daily_store_family")

print("✅ Created gold_daily_store_family (with time features)")

# Initial write of Gold fact table as managed UC table
base_fact.write.format("delta").mode("overwrite").saveAsTable("gold_daily_store_family")

print("✅ Gold table 'gold_daily_store_family' created.")


In [0]:
display(spark.table("cscie103_catalog.final_project.gold_daily_store_family"))


In [0]:
%sql
SELECT *
FROM cscie103_catalog.final_project.gold_daily_store_family
WHERE sales = 1
  AND is_holiday IS NOT NULL LIMIT 50;